In [ ]:
# !pip install kaggle
# api_token = {'username':'','key':''}
# import json
# with open('/content/kaggle.json', 'w') as file:
#     json.dump(api_token, file)
# !mkdir -p ~/.kaggle 
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d kritikseth/fruit-and-vegetable-image-recognition
# !unzip /content/fruit-and-vegetable-image-recognition.zip -d foodimages

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

In [ ]:
base_folder = '/content/foodimages'

for root, dirs, files in os.walk(base_folder, topdown=False):
   for name in dirs:
      categories = os.listdir(os.path.join(root, name))

ids = list(range(len(categories)))
cat_id = {cat:id for cat, id in zip(categories, ids)}

In [ ]:
train_path = base_folder + '/train'
test_path = base_folder + '/test'
valid_path = base_folder + '/validation'

train_paths = [train_path+'/'+category for category in categories]
test_paths = [test_path+'/'+category for category in categories]
valid_paths = [valid_path+'/'+category for category in categories]

In [ ]:
paths = []
for path in train_paths:
    paths.append([path+'/'+p for p in os.listdir(path)])

train_all_paths = [item for sublist in paths for item in sublist]

paths = []
for path in test_paths:
    paths.append([path+'/'+p for p in os.listdir(path)])

test_all_paths = [item for sublist in paths for item in sublist]

paths = []
for path in valid_paths:
    paths.append([path+'/'+p for p in os.listdir(path)])

valid_all_paths = [item for sublist in paths for item in sublist]

In [ ]:
food_train = [filename.split('/')[4] for filename in train_all_paths]
food_test = [filename.split('/')[4] for filename in test_all_paths]
food_valid = [filename.split('/')[4] for filename in valid_all_paths]

In [ ]:
train = pd.DataFrame({'File': train_all_paths, 'Label': food_train})
# train['Label'] = train['Label'].replace(cat_id)
train.to_csv('food_train.csv',index=False)

test = pd.DataFrame({'File': test_all_paths, 'Label': food_test})
# test['Label'] = test['Label'].replace(cat_id)
test.to_csv('food_test.csv',index=False)

valid = pd.DataFrame({'File': valid_all_paths, 'Label': food_valid})
# valid['Label'] = valid['Label'].replace(cat_id)
valid.to_csv('food_valid.csv',index=False)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255.0,
                                   rotation_range=10,
                                   zoom_range=0.1,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=None)

valid_datagen = ImageDataGenerator(rescale=1/255.0,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=None)

test_datagen = ImageDataGenerator(rescale=1/255.0)

In [ ]:
batch_size = 20
img_rows, img_cols = (100, 100)
input_shape = (img_rows, img_cols, 3)
dir = ''


train_aug = train_datagen.flow_from_dataframe(dataframe=train,
                                              directory=dir,
                                              x_col='File',
                                              y_col='Label',
                                              batch_size=batch_size,
                                              target_size=(img_rows, img_cols),
                                              class_mode='categorical',
                                              shuffle=True,
                                              color_mode='rgb',
                                              seed=123)

valid_aug = valid_datagen.flow_from_dataframe(dataframe=valid,
                                              directory=dir,
                                              x_col='File',
                                              y_col='Label',
                                              batch_size=batch_size,
                                              target_size=(img_rows, img_cols),
                                              class_mode='categorical',
                                              shuffle=True,
                                              color_mode='rgb',
                                              seed=123)

test_aug = test_datagen.flow_from_dataframe(dataframe=test,
                                            directory=dir,
                                            x_col='File',
                                            y_col='Label',
                                            batch_size=batch_size,
                                            target_size=(img_rows, img_cols),
                                            class_mode='categorical',
                                            shuffle=False,
                                            color_mode='rgb')

Found 3581 validated image filenames belonging to 36 classes.
Found 351 validated image filenames belonging to 36 classes.


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# VGG16

In [48]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape=input_shape, include_top = False, weights = 'imagenet')

In [49]:
for layer in base_model.layers:
    layer.trainable = False

In [50]:
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(36, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'categorical_crossentropy', metrics = ['acc'])

In [51]:
model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)     

In [ ]:
vgghist = model.fit(train_aug, validation_data=valid_aug, steps_per_epoch = 100, epochs = 15)

Epoch 1/15
  1/100 [..............................] - ETA: 0s - loss: 1.7963 - acc: 0.5000

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


 24/100 [======>.......................] - ETA: 52s - loss: 1.6490 - acc: 0.5333

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


100/100 [==============================] - 98s 984ms/step - loss: 1.6502 - acc: 0.5435 - val_loss: 1.2444 - val_acc: 0.6980
Epoch 2/15
100/100 [==============================] - 93s 931ms/step - loss: 1.6150 - acc: 0.5635 - val_loss: 1.1185 - val_acc: 0.7350
Epoch 3/15
100/100 [==============================] - 91s 913ms/step - loss: 1.5526 - acc: 0.5846 - val_loss: 1.0834 - val_acc: 0.7123
Epoch 4/15
100/100 [==============================] - 92s 917ms/step - loss: 1.5224 - acc: 0.5800 - val_loss: 1.1462 - val_acc: 0.6724
Epoch 5/15
100/100 [==============================] - 93s 926ms/step - loss: 1.4799 - acc: 0.5805 - val_loss: 1.0811 - val_acc: 0.7179
Epoch 6/15
100/100 [==============================] - 94s 945ms/step - loss: 1.4280 - acc: 0.5957 - val_loss: 1.0519 - val_acc: 0.7236
Epoch 7/15
100/100 [==============================] - 97s 971ms/step - loss: 1.4154 - acc: 0.6002 - val_loss: 0.9816 - val_acc: 0.7464
Epoch 8/15
100/100 [==============================] - 94s 943ms/st

In [ ]:
model.save('vgg16.hdf5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('vgg16.tflite', 'wb') as f:
  f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmpmmaw09zq/assets


# EffecientNet

In [ ]:
# !pip install efficientnet

In [43]:
import efficientnet.keras as efn

In [44]:
base_model = efn.EfficientNetB0(input_shape=input_shape, include_top = False, weights='imagenet')

In [45]:
for layer in base_model.layers:
    layer.trainable = False

In [46]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(36, activation='softmax')(x)
model_final = tf.keras.Model(base_model.input, predictions)

In [47]:
model_final.summary()

Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 50, 50, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 50, 50, 32)   128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 50, 50, 32)   0           stem_bn[0][0]                    
______________________________________________________________________________________

In [ ]:
model_final.compile(tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
eff_history = model_final.fit(train_aug, validation_data=valid_aug, epochs=5)

Epoch 1/5
 11/180 [>.............................] - ETA: 1:45 - loss: 3.6753 - accuracy: 0.0227

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


 22/180 [==>...........................] - ETA: 1:53 - loss: 3.7540 - accuracy: 0.0250

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


180/180 [==============================] - 159s 883ms/step - loss: 3.8525 - accuracy: 0.0251 - val_loss: 3.6318 - val_accuracy: 0.0228
Epoch 2/5
180/180 [==============================] - 156s 866ms/step - loss: 3.7017 - accuracy: 0.0260 - val_loss: 3.6443 - val_accuracy: 0.0256
Epoch 3/5
180/180 [==============================] - 151s 839ms/step - loss: 3.7020 - accuracy: 0.0271 - val_loss: 3.7705 - val_accuracy: 0.0256
Epoch 4/5
180/180 [==============================] - 149s 828ms/step - loss: 3.6973 - accuracy: 0.0271 - val_loss: 3.5852 - val_accuracy: 0.0256
Epoch 5/5
180/180 [==============================] - 149s 828ms/step - loss: 3.7320 - accuracy: 0.0274 - val_loss: 3.7931 - val_accuracy: 0.0256


In [ ]:
model_final.save('effnet_noscale.hdf5')
convert_bytes(get_file_size('effnet_noscale.hdf5'), 'MB')

File size: 176.345 Megabytes


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_final)
tflite_model = converter.convert()

with open('effnet.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp81s1m634/assets


INFO:tensorflow:Assets written to: /tmp/tmp81s1m634/assets
